# Redes Neurais - Projeto 3
# Modelos Generativos - Autoencoders Variacionais
---------------------
### Luis Filipe Menezes
#### RA: 164924

## 1. Objetivos:
Este caderno consiste na terceira entrega da disciplina de Redes Neurais realizada no programa de Pós Graduação em Ciência da Computação durante meu mestrado.

O projeto tem como objetivo:

- Selecionar 2 datasets (rotulados)

  - Treinar modelos VAEs:
  
  - Ajustar o melhor modelo (topologia) segundo a função de custo (conjunto validação)


- Explorar o espaço latente:

  - Gerar gráficos com a projeção do espaço latente em 2D (PCA)

  - Usar os rótulos na projeção.

- Algumas questões:

  1. Há formação de clusters no espaço latente?
  
  2. Há separação dos rótulos no espaço latente?
  
  3. A projeção ilustra quanto da variância?

- Adicional (opcional): Enviesar a formação do espaço latente com os exemplos rotulados


Além disso, tentarei utilizar o PyTorch nesse projeto. A implementação será feita se baseando neste [material](https://lightning.ai/docs/pytorch/stable/notebooks/course_UvA-DL/08-deep-autoencoders.html) e nesta outra implementação de [autoencoder com MLP](https://debuggercafe.com/implementing-deep-autoencoder-in-pytorch/).

Como o modelo que ele apresenta utiliza uma CNN no dataset Ciphar, como ainda não foi estudado CNNs irei utilizar uma MLP no dataset Fashion MNIST.


### Bibliotecas utilizadas:

In [1]:
!pip install pytorch_lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.6/831.6 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 50.2 MB/s eta 0:00:00


In [2]:
from torchvision.datasets import FashionMNIST
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from torchvision import transforms

## Dataset utilizado (Fashion MNIST)

#### Setup do sistema para garantir reprodutibilidade

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
pl.seed_everything(24)

INFO:lightning_fabric.utilities.seed:Seed set to 24


24

In [5]:
NUM_EPOCHS = 500
BATCH_SIZE = 256
LEARNING_RATE = 1e-3
LATENT_DIM = 16
INPUT_DIM = 28 * 28

In [6]:
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = FashionMNIST(root='./FMNIST_data', train=True, download=True, transform=transform)


100%|██████████| 26.4M/26.4M [00:02<00:00, 12.7MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 203kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.77MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 16.4MB/s]


In [7]:
test_dataset = FashionMNIST(root='./FMNIST_data', train=False, download=True, transform=transform)

In [8]:
train_set, val_set = torch.utils.data.random_split(train_dataset, [45000, 15000])


In [9]:
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, pin_memory=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False, num_workers=2)

In [10]:
from torchvision.utils import save_image
import os

# utility functions
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device
def make_dir():
    image_dir = 'FashionMNIST_Images'
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)
def save_decoded_image(img, epoch):
    img = img.view(img.size(0), 1, 28, 28)
    save_image(img, './FashionMNIST_Images/linear_ae_image{}.png'.format(epoch))

## Modelo em PyTorch

In [11]:
class VariationalAutoEncoder(nn.Module):
    def __init__(self, latent_dim: int, learning_rate: float):
        super(VariationalAutoEncoder, self).__init__()
        self.enc1 = nn.Linear(INPUT_DIM, 512)
        self.enc2 = nn.Linear(512, latent_dim*2)

        self.dec1 = nn.Linear(latent_dim,  512)
        self.dec2 = nn.Linear(512, INPUT_DIM)
        self.learning_rate = learning_rate
        self.criterion = nn.BCELoss(reduction="sum")


    def reparameterize(self, mu, logvar):
      """
      :param mu: mean from the encoder's latent space
      :param logvar: log variance from the encoder's latent space
      """
      std = torch.exp(0.5 * logvar)
      eps = torch.randn_like(std)
      sample = mu + (eps * std)
      return sample

    def forward(self, x):
        # encoding
        x = F.relu(self.enc1(x))
        x = self.enc2(x).view(-1, 2, LATENT_DIM)
        # get `mu` and `log_var`
        mu = x[:, 0, :] # the first feature values as mean
        log_var = x[:, 1, :] # the other feature values as variance
        # get the latent vector through reparameterization
        z = self.reparameterize(mu, log_var)

        # decoding
        x = F.relu(self.dec1(z))
        reconstruction = torch.sigmoid(self.dec2(x))
        return reconstruction, mu, log_var

    # def validation_step(self, batch, batch_idx):
    #     x, _ = batch
    #     x = x.view(x.size(0), -1)
    #     x_recon = self.forward(x)
    #     loss = self.criterion(x_recon, x)
    #     self.log('val_loss', loss)

    # def configure_optimizers(self):
    #     optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
    #     return optimizer

In [12]:
def final_loss(bce_loss, mu, logvar):
    """
    This function will add the reconstruction loss (BCELoss) and the
    KL-Divergence.
    KL-Divergence = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    :param bce_loss: recontruction loss
    :param mu: the mean from the latent vector
    :param logvar: log variance from the latent vector
    """
    BCE = bce_loss
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [13]:
model = VariationalAutoEncoder(latent_dim=LATENT_DIM, learning_rate=LEARNING_RATE).to(get_device())
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.BCELoss(reduction="sum") # específica para autoencoders

In [14]:
from tqdm import tqdm
def fit(model, dataloader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for i, data in tqdm(enumerate(dataloader), total=int(len(train_set)/dataloader.batch_size)):
        data, _ = data
        data = data.to(device)
        data = data.view(data.size(0), -1)
        optimizer.zero_grad()
        reconstruction, mu, logvar = model(data)
        bce_loss = criterion(reconstruction, data)
        loss = final_loss(bce_loss, mu, logvar)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    train_loss = running_loss/len(dataloader.dataset)
    return train_loss


def validate(model, dataloader, epoch, criterion):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=int(len(val_set)/dataloader.batch_size)):
            data, _ = data
            data = data.to(device)
            data = data.view(data.size(0), -1)
            reconstruction, mu, logvar = model(data)
            bce_loss = criterion(reconstruction, data)
            loss = final_loss(bce_loss, mu, logvar)
            running_loss += loss.item()

            # save the last batch input and output of every epoch
            if i == int(len(val_set)/dataloader.batch_size) - 1:
                num_rows = 8
                both = torch.cat((data.view(BATCH_SIZE, 1, 28, 28)[:8],
                                  reconstruction.view(BATCH_SIZE, 1, 28, 28)[:8]))
                # Ensure the directory exists before saving
                make_dir()
                save_image(both.cpu(), f"./FashionMNIST_Images/output{epoch}.png", nrow=num_rows)
    val_loss = running_loss/len(dataloader.dataset)
    return val_loss

In [ ]:
train_loss = []
val_loss = []

for epoch in range(NUM_EPOCHS):
    print(f"Epoch {epoch+1} of {NUM_EPOCHS}")
    train_epoch_loss = fit(model, train_loader, optimizer, criterion)
    val_epoch_loss = validate(model, val_loader, epoch, criterion)
    train_loss.append(train_epoch_loss)
    val_loss.append(val_epoch_loss)
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f"Val Loss: {val_epoch_loss:.4f}")

Epoch 1 of 500


100%|██████████| 175/175 [00:05<00:00, 33.97it/s]
59it [00:01, 39.70it/s]                        

Train Loss: 305.6155
Val Loss: 269.7706
Epoch 2 of 500



100%|██████████| 175/175 [00:05<00:00, 31.34it/s]
59it [00:01, 40.64it/s]                        

Train Loss: 262.6191
Val Loss: 258.8908
Epoch 3 of 500


...


100%|██████████| 175/175 [00:04<00:00, 36.10it/s]
59it [00:01, 38.68it/s]

Train Loss: 235.3154
Val Loss: 237.4843
Epoch 284 of 500



100%|██████████| 175/175 [00:05<00:00, 30.76it/s]
59it [00:01, 38.70it/s]                        

Train Loss: 235.3118
Val Loss: 237.2746
Epoch 285 of 500



 70%|███████   | 123/175 [00:03<00:01, 37.95it/s]

In [22]:
def test_image_reconstruction(net, testloader):
     for batch in testloader:
        img, _ = batch
        img = img.to(device)
        img = img.view(img.size(0), -1)
        outputs = net(img)
        outputs = outputs.view(outputs.size(0), 1, 28, 28).cpu().data
        save_image(outputs, 'fashionmnist_reconstruction.png')
        break

#### Treinamento

In [34]:
from matplotlib import pyplot as plt
# # train the network
# vae = VariationalAutoEncoder(latent_dim=LATENT_DIM, learning_rate=LEARNING_RATE)
# # net = vae.to(get_device())
# train_loss = train(vae, train_loader, NUM_EPOCHS, LEARNING_RATE)
# # plt.figure()
# # plt.plot(train_loss)
# # plt.title('Train Loss')
# # plt.xlabel('Epochs')
# # plt.ylabel('Loss')
# # plt.savefig('deep_ae_fashionmnist_loss.png')